In [125]:
import os
from pathlib import Path

path = Path.home() / "Desktop" / "CodeBasics" / "machine learning" / "krish naik" / "1st_project" / "data_science_project"
os.chdir(path)
print("Now in:", os.getcwd())


Now in: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project


In [126]:
from dataclasses import dataclass
from pathlib import Path

@dataclass

class data_ingestion_config:
    root_dir : Path
    source_url : str
    local_data_file : str
    unzip_dir : Path


In [127]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml , create_directories

class configmanager:
    def __init__(self, config_filepath = config_file_path , params_filepath = param_file_path , schema_filepath = schema_file_path):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> data_ingestion_config:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    


        dataingestion_config = data_ingestion_config(

            root_dir  = config.root_dir,
            source_url = config.source_url ,
            local_data_file = config.local_data_file ,
            unzip_dir = config.unzip_dir

        )

        return dataingestion_config

In [128]:
import urllib.request as requests
import os
from src.datascience import logger
import zipfile 

import mimetypes
import requests

class data_ingestion:

    def __init__(self , config: data_ingestion_config):
        self.config = config


    def download_file(self):
     if not os.path.exists(self.config.local_data_file):
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.117 Safari/537.36"
        }

        response = requests.get(self.config.source_url, headers=headers, stream=True, allow_redirects=True)

        # Check response
        content_type = response.headers.get("Content-Type", "")
        if "zip" not in content_type:
            raise ValueError(f"Expected a zip file but got content-type: {content_type}")

        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
        with open(self.config.local_data_file, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        logger.info(f"Downloaded ZIP to {self.config.local_data_file}")

     else:
        logger.info(f"{self.config.local_data_file} already exists")    




    # def download_file(self):
    #     if not os.path.exists(self.config.local_data_file):
    #         filename , header = requests.urlretrieve(
    #             url = self.config.source_url,
    #             filename=self.config.local_data_file
    #         )
    #         logger.info(f'{filename} download with following info {header} !')

    #     else:
    #         logger.info(f'file already exists')    

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file , 'r')  as zip_ref:
            zip_ref.extractall(unzip_path)    

In [129]:
try:
    config = configmanager() 
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = data_ingestion(config=data_ingestion_config) 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
   raise e    





[2025-06-29 13:55:10,581 : INFO : common : YAML file: config/config.yaml loaded successfully.]
[2025-06-29 13:55:10,583 : INFO : common : YAML file: params.yaml loaded successfully.]
[2025-06-29 13:55:10,584 : INFO : common : YAML file: schema.yaml loaded successfully.]
[2025-06-29 13:55:10,584 : INFO : common : Directory created at: artifacts]
[2025-06-29 13:55:10,584 : INFO : common : Directory created at: artifacts/data_ingestion]
[2025-06-29 13:55:12,833 : INFO : 2706430546 : Downloaded ZIP to artifacts/data_ingestion/data.zip]
